In [1]:
import torch
import random
import zipfile

In [2]:
with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars=f.read().decode('utf-8')
corpus_chars[:100]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每天在想想想想著你\n这样的甜蜜\n让我开始乡相信命运\n感谢地心引力\n让我碰到你\n漂亮的让我面红的可爱女人\n温柔的让我心疼的可'

In [3]:
corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars=corpus_chars[0:10000]

In [4]:
idx_to_char=list(set(corpus_chars))
char_to_idx=dict([( char,i ) for i , char in enumerate(idx_to_char)])
vocab_size=len(char_to_idx)
vocab_size

1027

In [5]:
corpus_indices=[char_to_idx[char] for char in corpus_chars]
sample=corpus_indices[:20]
print('chars : ',''.join([idx_to_char[idx] for idx in sample]))
print('indices : ',sample)

chars :  想要有直升机 想要和你飞到宇宙去 想要和
indices :  [592, 73, 45, 902, 470, 315, 80, 592, 73, 329, 768, 652, 590, 856, 148, 39, 80, 592, 73, 329]


In [6]:
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
    num_examples=(len(corpus_indices)-1) // num_steps
    epoch_size=num_examples // batch_size
    example_indices=list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data (pos):
        return corpus_indices[pos:pos+num_steps]
    
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    for i in range(epoch_size):
        i = i* batch_size
        batch_indices=example_indices[i:i+batch_size]
        X=[_data(j* num_examples) for j in batch_indices]
        Y=[_data(j*num_examples+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device),torch.tensor(Y,dtype=torch.float32,device=device)

In [7]:
my_seq=list(range(30))
for X,Y in data_iter_random(my_seq,batch_size=2,num_steps=6):
    print('X : ',X,'\nY : ', Y,'\n')

X :  tensor([[12., 13., 14., 15., 16., 17.],
        [ 0.,  1.,  2.,  3.,  4.,  5.]]) 
Y :  tensor([[13., 14., 15., 16., 17., 18.],
        [ 1.,  2.,  3.,  4.,  5.,  6.]]) 

X :  tensor([[ 4.,  5.,  6.,  7.,  8.,  9.],
        [ 8.,  9., 10., 11., 12., 13.]]) 
Y :  tensor([[ 5.,  6.,  7.,  8.,  9., 10.],
        [ 9., 10., 11., 12., 13., 14.]]) 



In [8]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        corpus_indices=torch.tensor(corpus_indices,dtype=torch.float32,device=device)
        data_len=len(corpus_indices)
        batch_len=data_len//batch_size
        indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
        epoch_size=(batch_len-1) // num_steps
        for i in range(epoch_size):
            i=i*num_steps
            X=indices[:,i:i+num_steps]
            Y=indices[:,i+1:i+num_steps+1]
            yield X,Y

In [9]:
for X,Y in data_iter_consecutive(my_seq,batch_size=2,num_steps=6):
    print('X : ',X,'\nY: ',Y,'\n')

X :  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]]) 
Y:  tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]]) 

X :  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]]) 
Y:  tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]]) 



In [13]:
for X,Y in data_iter_consecutive(my_seq,batch_size=2,num_steps=6):
    print('Y',Y)
    print('Y transpose ',torch.transpose(Y,0,1))
    print('Y contiguous ',torch.transpose(Y,0,1).contiguous())
    print('Y view ',torch.transpose(Y,0,1).contiguous().view(-1))

Y tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]])
Y transpose  tensor([[ 1., 16.],
        [ 2., 17.],
        [ 3., 18.],
        [ 4., 19.],
        [ 5., 20.],
        [ 6., 21.]])
Y contiguous  tensor([[ 1., 16.],
        [ 2., 17.],
        [ 3., 18.],
        [ 4., 19.],
        [ 5., 20.],
        [ 6., 21.]])
Y view  tensor([ 1., 16.,  2., 17.,  3., 18.,  4., 19.,  5., 20.,  6., 21.])
Y tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]])
Y transpose  tensor([[ 7., 22.],
        [ 8., 23.],
        [ 9., 24.],
        [10., 25.],
        [11., 26.],
        [12., 27.]])
Y contiguous  tensor([[ 7., 22.],
        [ 8., 23.],
        [ 9., 24.],
        [10., 25.],
        [11., 26.],
        [12., 27.]])
Y view  tensor([ 7., 22.,  8., 23.,  9., 24., 10., 25., 11., 26., 12., 27.])
